# Tutorial 2 - Hilbert spaces

Hilbert spaces are complete inner product spaces. Within all inverse problems considered we will assume that both the model and data space are Hilbert spaces. This is a restriction, but one that is difficult to avoid in practice. The data space in any real inverse problem is necessarily finite dimensional and so can always be identified with $\mathbb{R}^{n}$ for some 
suitable $n \in \mathbb{N}$. In most cases of interest, however, the model space will be infinite-dimensional. Computationally we cannot deal directly with infinite-dimensional spaces, but we can ensure that we work with convergent discretisations. This means, roughly speaking, that the underlying mathematical problem is well-defined, and that our numerical solutions can be shown to converge to this solution as the size of our discretised model space is suitably increased.

Within this tutorial we look into the representation of Hilbert spaces within the ```pygeoinf``` library.


Within ```pygeoinf```, Hilbert spaces are represented by the ```HilbertSpace``` class. This class defined only the *structure* of the Hilbert space but requires that the underlying elements have been otherwise defined. To define an instance of the ```HilbertSpace``` class the user must provide the following data:

- The dimension of the space. 
- A mapping from element of the space to a chosen co-ordinate representation. 
- A mapping from this co-ordinate representation to elements of the space. 
- The inner product for the space. 
- A mapping from an element of the space to the associated dual vector, as provided by the Riesz representation theorem. 
- A mapping from an element of the dual space into the space, as provided by the Riesz representation theorem. 

In the case of infinite-dimensional spaces, the dimension is understood as being that of the chosen approximating space. 




### Euclidean space

These ideas can be illistrated by the case of $n$-dimensional Euclidean space which is implemented in the library as the class ```EuclideanSpace```
which is derived from ```HilbertSpace```:

In [42]:
import numpy as np
import pygeoinf as inf

# Set up Euclidean space. 
dim = 4
X = inf.EuclideanSpace(dim)

# Print its dimension. 
print(f'Dimension of the vector space = {X.dim}')

Dimension of the vector space = 4


Elements of ```EuclideanSpace``` are just ```numpy``` vectors. In the following we generate a random element and get its components. For ```EuclideanSpace``` the vector and the vector of its components are the same. Note that the ```random``` method for ```HilbertSpaces``` internally generates a component vector drawn from a standard Gaussian distribution and then maps this to an element of the space. It is only meant to provide a quick method for generating elements of the space for testing. 

In [43]:
# Generate a random element of the space and print its value. 
x = X.random()
print(f'The vector is equal to: {x}')

# Now get its component vector, which in this case is the same thing. 
c = X.to_components(x)
print(f'The vector\'s components are equal to: {c}')

The vector is equal to: [-0.53770022  1.27734382  0.90059497 -1.00475177]
The vector's components are equal to: [-0.53770022  1.27734382  0.90059497 -1.00475177]


If needed, we can access the zero vector is the space as follows:

In [44]:
# Generate a the zero vector -- note that this is a property of the class. 
x = X.zero
print(f'The vector is equal to: {x}')

# Now get its component vector, which in this case is the same thing. 
c = X.to_components(x)
print(f'The vector\'s components are equal to: {c}')

The vector is equal to: [0. 0. 0. 0.]
The vector's components are equal to: [0. 0. 0. 0.]


Given two element of the space, we can compute their inner product:

In [47]:
x1 = X.random()
x2 = X.random()
print(f'The first vector is: {x1} ')
print(f'The first vector is: {x2} ')
print(f'Their inner product is equal to: {X.inner_product(x1, x2)}')

The first vector is: [ 0.91525296 -1.89182274  0.75049115  2.17347311] 
The first vector is: [ 0.78448083  0.52411084  0.88085966 -0.40059015] 
Their inner product is equal to: -0.4831209402059937


The inner product defines a norm, and we can access this as follows:

In [46]:
x = X.random()
print(f'The vector is equal to: {x}')
print(f'The vector has norm: {X.norm(x)}')
print(f'The vector has norm: {X.squared_norm(x)}')

The vector is equal to: [ 1.04734752 -0.51694196  0.49552312 -0.6762424 ]
The vector has norm: 1.437710942159453
The vector has norm: 2.067012753205022


We can use these methods to verify that standard Hilbert space inequalities are satified. 

In [ ]:
x1 = X.random()
x2 = X.random()
print(f"The first vector is: {x1} ")
print(f"The first vector is: {x2} ")

print(
    f"The triangle inequality, |x1 + x2| <= |x1| + |x2|, is {X.norm(x1+x2) <= X.norm(x1) + X.norm(x2)}."
)

print(
    f"The Cauchy-Scwharz inequality, |(x1,x2)| <= |x1||x2|, is {np.abs(X.inner_product(x1,x2)) <= X.norm(x1) * X.norm(x2)}."
)



The first vector is: [-0.18340147 -0.71974559 -0.52329397  0.77292779] 
The first vector is: [0.83683643 0.51411009 0.25027152 0.87608565] 
The triangle inequality, |x1 + x2| <= |x1| + |x2|, is True.
The Cauchy-Scwharz inequality, |(x1,x2)| <= |x1||x2|, is True.


For ```HilbertSpaces``` whose elements have the standard vector operations already defined, as is the case here, we can directly add, subtract or scalar multiply them. 
More generally, methods are provided for these operations as shown below:

In [ ]:
x1 = X.random()
x2 = X.random()
print(f"\nThe first vector is: {x1} ")
print(f"The first vector is: {x2} ")

x3 = X.add(x1,x2)
print(f"The sum of the vectors is: {x3} ")

print(f"The first vector times by 2 is is: {X.multiply(2,x1)} ")


The first vector is: [-0.58084188 -1.01476081 -1.44230369 -1.661966  ] 
The first vector is: [-0.10650031  0.14823766  0.82345458 -0.58650551] 
The sum of the vectors is: [-0.68734219 -0.86652315 -0.61884911 -2.24847151] 
The first vector times by 2 is is: [-1.16168376 -2.02952163 -2.88460738 -3.32393201] 


When a ```HilbertSpace``` is constructed custom implementation of these vector operations can be provided, and internally these are all that are used. 